# xmitgcm llcreader demo

This notebooks contains a demonstration of reading and visulizing data from the
[NAS ECCO Data Portal](https://data.nas.nasa.gov/ecco/).
It makes use of the following software libraries:

- [xmitgcm](https://xmitgcm.readthedocs.io/en/latest/llcreader.html): provides the `llcreader` module which makes all of this work
- [xarray](http://xarray.pydata.org/en/latest/): the basic data structures and computational library for the datasets 
- [dask](https://dask.org/): the parallel computing library which enables lazy representations of huge arrays
- [holoviews](http://holoviews.org/): interactive visualizations

We start by importing the necessary libraries

In [1]:
#uncomment below to quickly install xmitgcm 
# !pip install fsspec zarr git+https://github.com/xgcm/xmitgcm.git

In [ ]:
import xmitgcm.llcreader as llcreader
import xmitgcm
import matplotlib.pyplot as plt
import dask
import numpy as np
from dask.distributed import Client
import xarray as xr
import requests
import time

dask.config.set(**{'array.slicing.split_large_chunks': False})

from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 11
Total threads: 11,Total memory: 18.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59844,Workers: 11
Dashboard: http://127.0.0.1:8787/status,Total threads: 11
Started: Just now,Total memory: 18.00 GiB
Comm: tcp://127.0.0.1:59870,Total threads: 1
Dashboard: http://127.0.0.1:59871/status,Memory: 1.64 GiB
Nanny: tcp://127.0.0.1:59847,


### Initialize Model Object

[xmitgcm.llcreader]() contains pre-configured references to known LLC model configurations.
We create one of these references as follows:

In [3]:
def drop_non_essential_coords(ds): 
    all_dims = list(ds.dims)
    all_coords = list(ds.coords)
    non_ess_coords = list(set(all_coords) - set(all_dims))
    return ds.drop_vars(non_ess_coords)

# def select_from_lat_lon(ds, lat, lon): 
#     distance = np.sqrt((ds["XC"] - lat)**2 + (ds["YC"] - lon)**2 )
#     return distance.argmin()

def get_lat_lon_idx(lat, lon, XC, YC): 
    distance = np.sqrt(((XC - lon )**2) + ((YC - lat)**2))
    return distance.argmin(dim = ["face", "i", "j"])

def extract_point(indices, ds): 
    return ds.sel(face = indices["face"], i = indices["i"], j = indices["j"])
def extract_lat_lon(indices, XC, YC):
    lon = extract_point(indices, XC).values
    lat = extract_point(indices, YC).values
    return lon, lat
def attempt_download(ds, max_attempts = 5, wait_seconds = 2):
    
    for attempt in range(1, max_attempts + 1):
        try:
            print(f"Attempt {attempt}...")
            ds_compute = drop_non_essential_coords(ds).compute()
            print("Download succeeded.")

            return ds_compute
        except Exception as e:
            print(f"Attempt {attempt} failed: {e}")
            if attempt < max_attempts:
                time.sleep(wait_seconds)
            else:
                print("All attempts failed. Giving up.")
       

In [4]:
LLC2160 = llcreader.ECCOPortalLLC2160Model()
LLC4320 = llcreader.ECCOPortalLLC4320Model()

In [5]:
ds = LLC2160.get_dataset(varnames=['Theta', 'Salt', "Depth", "PhiBot"], k_chunksize=90) 

In [6]:
XC = attempt_download(ds["XC"])
YC = attempt_download(ds["YC"])

Attempt 1...
Download succeeded.
Attempt 1...


2025-05-16 12:01:06,369 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/dis

Attempt 1 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data')


2025-05-16 12:01:06,892 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/dis

Attempt 2...
Attempt 2 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data')


...

Attempt 3...
Attempt 3 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data')


2025-05-16 12:01:10,851 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/dis

Attempt 4...
Attempt 4 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data')


2025-05-16 12:01:13,046 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/grid/YC.data')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/dis

Attempt 5...
Download succeeded.


In [7]:
lats = [26.4971,26.4901]
lons = [-76.7510, -76.5050]
indices = []
names = ["nearWB2LZ", "nearWB3LZ"]

for i in range(2):
     indices += [get_lat_lon_idx(lats[i], lons[i], XC, YC)]

In [8]:
locnum = 1
ds_loc = extract_point(indices[locnum], ds).isel(time = slice(0, 10))

In [9]:
depth = attempt_download(ds_loc["Depth"])

Attempt 1...
Download succeeded.


In [10]:
PhiBot = attempt_download(ds_loc["PhiBot"])

Attempt 1...


2025-05-16 12:03:20,712 - distributed.protocol.pickle - ERROR - Failed to serialize 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092320/PhiBot.0000092320.data.shrunk').
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/distributed/worker.py", line 2997, in apply_function_simple
    result = function(*args, **kwargs)
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/distributed/worker.py", line 2907, in execute_task
    return func(*map(execute_task, args))
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/distributed/worker.py", line 2907, in execute_task
    return func(*map(execute_task, args))
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/distributed/worker.py", line 2907, in execute_task
    return func(*map(execute_task, args))
  [Previous line repeated 2 more times]
  File "/Users/anthonymeza/min

Attempt 1 failed: ClientResponseError(RequestInfo(url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092320/PhiBot.0000092320.data.shrunk'), method='GET', headers=<CIMultiDictProxy('Host': 'data.nas.nasa.gov', 'Range': 'bytes=109163716-140637811', 'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br', 'User-Agent': 'Python/3.10 aiohttp/3.9.5', 'Cookie': 'session=expiry=1747411999182033')>, real_url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092320/PhiBot.0000092320.data.shrunk')), (), status=502, message='Bad Gateway', headers=<CIMultiDictProxy('Server': 'nginx', 'Date': 'Fri, 16 May 2025 16:03:20 GMT', 'Content-Type': 'text/html', 'Content-Length': '166', 'Connection': 'keep-alive', 'Content-Security-Policy': "default-src 'self' http: https: data: blob: 'unsafe-inline'")>)


2025-05-16 12:03:22,065 - distributed.protocol.pickle - ERROR - Failed to serialize 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092400/PhiBot.0000092400.data.shrunk').
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/distributed/worker.py", line 2997, in apply_function_simple
    result = function(*args, **kwargs)
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/distributed/worker.py", line 2907, in execute_task
    return func(*map(execute_task, args))
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/distributed/worker.py", line 2907, in execute_task
    return func(*map(execute_task, args))
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/distributed/worker.py", line 2907, in execute_task
    return func(*map(execute_task, args))
  [Previous line repeated 2 more times]
  File "/Users/anthonymeza/min

Attempt 2...


...

2025-05-16 12:03:41,787 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092160/PhiBot.0000092160.data.shrunk.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092160/PhiBot.0000092160.data.shrunk')

The above exception was the direct cause of the following exception:

Traceback (most recent call

Attempt 4...


2025-05-16 12:03:43,829 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092560/PhiBot.0000092560.data.shrunk.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092560/PhiBot.0000092560.data.shrunk')

The above exception was the direct cause of the following exception:

Traceback (most recent call

Attempt 4 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092880/PhiBot.0000092880.data.shrunk')
Attempt 5...
Attempt 5 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092240/PhiBot.0000092240.data.shrunk')
All attempts failed. Giving up.


2025-05-16 12:03:46,161 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092560/PhiBot.0000092560.data.shrunk.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092560/PhiBot.0000092560.data.shrunk')

The above exception was the direct cause of the following exception:

Traceback (most recent call

In [ ]:
ds_vars = attempt_download(ds_loc[['Theta', 'Salt']])

Attempt 1...


2025-05-16 12:05:53,866 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092720/Theta.0000092720.data.shrunk.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092720/Theta.0000092720.data.shrunk')

The above exception was the direct cause of the following exception:

Traceback (most recent call l

Attempt 1 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092400/Theta.0000092400.data.shrunk')


2025-05-16 12:05:54,141 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092640/Theta.0000092640.data.shrunk.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092640/Theta.0000092640.data.shrunk')

The above exception was the direct cause of the following exception:

Traceback (most recent call l

Attempt 2...


...

Attempt 2 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092880/Theta.0000092880.data.shrunk')
Attempt 3...


2025-05-16 12:05:59,189 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092880/Theta.0000092880.data.shrunk.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092880/Theta.0000092880.data.shrunk')

The above exception was the direct cause of the following exception:

Traceback (most recent call l

Attempt 3 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092880/Theta.0000092880.data.shrunk')
Attempt 4...


2025-05-16 12:06:01,607 - distributed.protocol.pickle - ERROR - Failed to serialize https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092800/Theta.0000092800.data.shrunk.
Traceback (most recent call last):
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 427, in _info
    await _file_info(
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/fsspec/implementations/http.py", line 819, in _file_info
    r.raise_for_status()
  File "/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 502, message='Bad Gateway', url=URL('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092800/Theta.0000092800.data.shrunk')

The above exception was the direct cause of the following exception:

Traceback (most recent call l

Attempt 4 failed: FileNotFoundError('https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_2160/compressed/0000092320/Theta.0000092320.data.shrunk')
Attempt 5...


In [12]:
ds_vars